In [1]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
from folium.plugins import HeatMap

### Read data

In [55]:
pd.set_option('display.max_rows', 500)

df = pd.read_parquet('/home/foolingeng/ruien/data/grab-posisi/sg-with-pop-hdb-amenities/part-00000-8bbff892-97d2-4011-9961-703e38972569_new.parquet')
df.dropna()
df['pingtimestamp'] = pd.to_datetime(df['pingtimestamp'], unit='s')
print(df["pingtimestamp"].min(), df["pingtimestamp"].max(), df.info()) # from 2019-04-08 to 2019-04-21

pd.reset_option('all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3034553 entries, 0 to 3034552
Data columns (total 35 columns):
 #   Column            Dtype         
---  ------            -----         
 0   trj_id            object        
 1   driving_mode      object        
 2   osname            object        
 3   pingtimestamp     datetime64[ns]
 4   rawlat            float64       
 5   rawlng            float64       
 6   speed             float64       
 7   bearing           int64         
 8   accuracy          float64       
 9   population_2020   float64       
 10  real_value        float64       
 11  school            object        
 12  school_lat        float64       
 13  school_lng        float64       
 14  school_dst        float64       
 15  supermarket       object        
 16  supermarket_lat   float64       
 17  supermarket_lng   float64       
 18  supermarket_dst   float64       
 19  MRT               object        
 20  MRT_lat           float64       
 21  MRT_lng 

/home/foolingeng/miniconda3/envs/posisi/lib/python3.8/site-packages/pandas/_config/config.py:645: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
/home/foolingeng/miniconda3/envs/posisi/lib/python3.8/site-packages/pandas/_config/config.py:645: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [ ]:
df.loc[df['trj_id'] == "70014"]

In [56]:
df = df.sort_values(['pingtimestamp'], ascending=True).sort_values('trj_id')
df.reset_index()
df.columns


Index(['trj_id', 'driving_mode', 'osname', 'pingtimestamp', 'rawlat', 'rawlng',
       'speed', 'bearing', 'accuracy', 'population_2020', 'real_value',
       'school', 'school_lat', 'school_lng', 'school_dst', 'supermarket',
       'supermarket_lat', 'supermarket_lng', 'supermarket_dst', 'MRT',
       'MRT_lat', 'MRT_lng', 'MRT_dst', 'parks', 'parks_lat', 'parks_lng',
       'parks_dst', 'hawker', 'hawker_lat', 'hawker_lng', 'hawker_dst',
       'shoppingmall', 'shoppingmall_lat', 'shoppingmall_lng',
       'shoppingmall_dst'],
      dtype='object')

In [57]:
trip_info = df.groupby('trj_id').agg(
    {'trj_id': 'first', 'driving_mode': 'first', 'osname': 'first'})

# grouped_lists = df.groupby("trj_id").apply(lambda x: [list(x['pingtimestamp']), list(x['rawlat']), list(x['rawlng'])]).apply(pd.Series)
# grouped_lists.columns = ['pingtimestamp', 'rawlat', 'rawlng']
# grouped_lists=grouped_lists.reset_index()

grouped = df.groupby("trj_id")
trip_info['start_time'] = grouped['pingtimestamp'].aggregate(min)
trip_info['end_time'] = grouped['pingtimestamp'].aggregate(max)
trip_info['eta'] = trip_info['end_time'] - trip_info['start_time']
trip_info

,trj_id,driving_mode,osname,start_time,end_time,eta
trj_id,,,,,,
10,10,car,android,2019-04-09 11:25:54,2019-04-09 11:44:54,0 days 00:19:00
100,100,car,ios,2019-04-16 02:44:24,2019-04-16 03:00:37,0 days 00:16:13
1000,1000,car,ios,2019-04-11 00:49:45,2019-04-11 01:03:22,0 days 00:13:37
10001,10001,car,android,2019-04-17 11:56:50,2019-04-17 12:17:05,0 days 00:20:15
10004,10004,car,ios,2019-04-09 03:12:39,2019-04-09 03:26:37,0 days 00:13:58
...,...,...,...,...,...,...
997,997,car,ios,2019-04-12 07:36:26,2019-04-12 07:56:08,0 days 00:19:42
9970,9970,car,android,2019-04-18 09:25:42,2019-04-18 09:54:53,0 days 00:29:11
998,998,car,ios,2019-04-18 22:30:33,2019-04-18 22:51:01,0 days 00:20:28


In [ ]:
import geopy.distance
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=16)

def calc_distance(row):
    distance = 0
    lat = row['rawlat']
    lng = row['rawlng']
    for i in range(0,len(lat)-1):
        distance += geopy.distance.geodesic((lat[i],lng[i]),(lat[i+1],lng[i+1])).km
    return distance

df_dist = df.loc[:, ['rawlat', 'rawlng']]
df_dist['rawlat_lag'] = df_dist['rawlat'].shift()
df_dist['rawlng_lag'] = df_dist['rawlng'].shift()
df_dist = df_dist.dropna()
# df_dist[abs(df_dist['rawlat_lag']) > 90]
df_dist['dist'] = df_dist.parallel_apply(lambda x: geopy.distance.geodesic(
    (x[2], x[3]), (x[0], x[1])), axis=1)


,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,population_2020,...,hawker,hawker_lat,hawker_lng,hawker_dst,shoppingmall,shoppingmall_lat,shoppingmall_lng,shoppingmall_dst,rawlat_lag,rawlng_lag
339331,10,car,android,2019-04-09 11:26:57,1.301633,103.801078,12.052279,14,6.0,23.590821,...,Tanglin Halt Market,1.300558,103.797719,391.916041,Rochester Mall,1.305408,103.788447,1465.158072,1.298342,103.901052
336286,10,car,android,2019-04-09 11:27:29,1.305926,103.801726,12.384522,16,8.0,23.590821,...,Commonwealth Crescent Market,1.306995,103.800357,193.099755,Rochester Mall,1.305408,103.788447,1477.007519,1.349808,103.925702
328125,10,car,android,2019-04-09 11:27:32,1.306247,103.801838,13.269312,19,8.0,23.590821,...,Commonwealth Crescent Market,1.306995,103.800357,184.455929,Rochester Mall,1.305408,103.788447,1491.205170,1.347057,103.842546
334513,10,car,android,2019-04-09 11:27:33,1.306360,103.801877,13.332310,19,8.0,23.590821,...,Commonwealth Crescent Market,1.306995,103.800357,183.135839,Rochester Mall,1.305408,103.788447,1496.366974,1.297399,103.898482
337879,10,car,android,2019-04-09 11:33:03,1.324524,103.813553,8.059647,11,4.0,27.918119,...,Adam Road Food Centre,1.324083,103.814182,85.431909,Tanglin Mall,1.304853,103.823883,2469.349462,1.374718,103.934300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325792,9984,car,ios,2019-04-20 01:51:27,1.279960,103.818593,14.797242,121,8.0,48.085619,...,Blk 11 Telok Blangah Crescent,1.277367,103.818642,288.260981,IMM,1.273329,103.811852,1050.854192,1.355723,103.857017
313622,9984,car,ios,2019-04-20 01:42:40,1.323625,103.745150,16.867191,101,12.0,25.020748,...,Market Street Food Centre,1.343516,103.737851,2354.767378,Big Box,1.331833,103.744879,912.641320,1.284673,103.837348
319301,9984,car,ios,2019-04-20 01:41:57,1.327250,103.738366,21.372160,105,12.0,25.020748,...,Market Street Food Centre,1.343516,103.737851,1808.694125,JCube,1.333238,103.740191,695.707317,1.324477,103.898699
326776,9984,car,ios,2019-04-20 01:40:22,1.326484,103.725512,9.752789,158,16.0,62.929828,...,Taman Jurong Market & Food Centre,1.334589,103.721520,1004.168643,Taman Jurong Shopping Centre,1.334845,103.720462,1085.569457,1.353402,103.788052
